In [ ]:
# ==============================
# 🔹 CGM-Only LSTM & GRU (without extra food related data)
# ==============================

!pip install tensorflow scikit-learn matplotlib pandas --quiet

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# ==============================
# 1️⃣ Load and Prepare Data
# ==============================
file_path = "/content/drive/MyDrive/diabetes/Subject 1_3hr.csv"  # ✅ Update if needed
data = pd.read_csv(file_path)

if 'CGM' not in data.columns:
    raise ValueError("❌ 'CGM' column not found in dataset!")

# Smooth the CGM signal (rolling mean)
data['CGM_smooth'] = data['CGM'].rolling(window=3, center=True).mean()
data = data.dropna(subset=['CGM_smooth']).reset_index(drop=True)

# Normalize
scaler = MinMaxScaler()
scaled_cgm = scaler.fit_transform(data[['CGM_smooth']])

# ==============================
# 2️⃣ Create Sequence Data
# ==============================
def create_sequences(dataset, window=20):
    X, y = [], []
    for i in range(len(dataset) - window):
        X.append(dataset[i:i + window])
        y.append(dataset[i + window])
    return np.array(X), np.array(y)

window_size = 20
X, y = create_sequences(scaled_cgm, window_size)

# Split data sequentially (no shuffle)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"✅ Data prepared: X_train={X_train.shape}, X_test={X_test.shape}")

# ==============================
# 3️⃣ Define Models
# ==============================

# LSTM
lstm_model = Sequential([
    LSTM(128, activation='tanh', return_sequences=True, input_shape=(window_size, 1)),
    Dropout(0.2),
    LSTM(64, activation='tanh', return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(0.001), loss='mse')

# GRU
gru_model = Sequential([
    GRU(128, activation='tanh', return_sequences=True, input_shape=(window_size, 1)),
    Dropout(0.2),
    GRU(64, activation='tanh', return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1)
])
gru_model.compile(optimizer=Adam(0.001), loss='mse')

# ==============================
# 4️⃣ Train (100 Epochs)
# ==============================
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("\n🚀 Training LSTM...")
history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs=100, batch_size=8, validation_split=0.2,
    verbose=1, callbacks=[es]
)

print("\n🚀 Training GRU...")
history_gru = gru_model.fit(
    X_train, y_train,
    epochs=100, batch_size=8, validation_split=0.2,
    verbose=1, callbacks=[es]
)

# ==============================
# 5️⃣ Evaluate Models
# ==============================
def evaluate_model(model, X_test, y_test, name):
    preds = model.predict(X_test).flatten()
    r2 = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    print(f"\n📊 {name} Results:")
    print(f"R²   = {r2:.4f}")
    print(f"RMSE = {rmse:.6f}")
    print(f"MAE  = {mae:.6f}")
    return {"Model": name, "R2": r2, "RMSE": rmse, "MAE": mae}

lstm_results = evaluate_model(lstm_model, X_test, y_test, "LSTM")
gru_results = evaluate_model(gru_model, X_test, y_test, "GRU")

# ==============================
# 6️⃣ Plot Predictions
# ==============================
plt.figure(figsize=(12,6))
plt.plot(y_test, label="True CGM", linewidth=2)
plt.plot(lstm_model.predict(X_test), label=f"LSTM (R²={lstm_results['R2']:.2f})", alpha=0.8)
plt.plot(gru_model.predict(X_test), label=f"GRU (R²={gru_results['R2']:.2f})", alpha=0.8)
plt.title("📈 CGM Prediction (True vs Predicted)")
plt.xlabel("Time Steps")
plt.ylabel("Scaled CGM")
plt.legend()
plt.show()

# ==============================
# 7️⃣ Save Results Only
# ==============================
save_dir = "/content/drive/MyDrive/diabetes/results/"
os.makedirs(save_dir, exist_ok=True)

results_df = pd.DataFrame([lstm_results, gru_results])
results_path = os.path.join(save_dir, "CGM_LSTM_GRU_results.csv")
results_df.to_csv(results_path, index=False)

print("\n✅ Results saved successfully at:", results_path)
print(results_df)


In [ ]:
# ==============================
# 🔹 Traditional ML Models for CGM-Only Data(without extra food related data)
# ==============================

!pip install lightgbm scikit-learn pandas --quiet

import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import lightgbm as lgb

# ==============================
# 1️⃣ Reuse Preprocessed Data
# ==============================
# Make sure you’ve run the previous LSTM-GRU section first.
# It already defines: X_train, X_test, y_train, y_test

print(f"✅ Using existing data shapes: X_train={X_train.shape}, X_test={X_test.shape}")

# Flatten sequence data for traditional models
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# ==============================
# 2️⃣ Define Traditional Models
# ==============================
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=200, random_state=42),
    "HistGB": HistGradientBoostingRegressor(random_state=42),
    "LightGBM": lgb.LGBMRegressor(n_estimators=200, random_state=42)
}

results = []

# ==============================
# 3️⃣ Train and Evaluate
# ==============================
for name, model in models.items():
    print(f"\n🚀 Training {name}...")
    model.fit(X_train_flat, y_train)
    preds = model.predict(X_test_flat)
    r2 = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    results.append({"Model": name, "R2": r2, "RMSE": rmse, "MAE": mae})
    print(f"{name}: R2={r2:.3f}, RMSE={rmse:.4f}, MAE={mae:.4f}")

# ==============================
# 4️⃣ Save Results to Same File
# ==============================
save_dir = "/content/drive/MyDrive/diabetes/results/"
os.makedirs(save_dir, exist_ok=True)

results_path = os.path.join(save_dir, "CGM_LSTM_GRU_results.csv")

# Append new results to the file created by the LSTM/GRU section
if os.path.exists(results_path):
    existing_results = pd.read_csv(results_path)
    final_results = pd.concat([existing_results, pd.DataFrame(results)], ignore_index=True)
else:
    final_results = pd.DataFrame(results)

final_results.to_csv(results_path, index=False)

print("\n✅ All Results Saved at:", results_path)
print(final_results)


In [ ]:
import numpy as np, tensorflow as tf
print("✅ NumPy:", np.__version__)
print("✅ TensorFlow:", tf.__version__)



In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 1_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()


In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from google.colab import drive
# ================================
# 💾 Mount Drive
# ================================
drive.mount('/content/drive')


# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 2_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 3_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 4_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 5_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 6_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 7_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 8_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 9_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 10_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 11_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 12_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 13_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 14_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 15_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 16_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 17_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 18_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 19_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 20_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 21_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 22_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 23_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 24_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()

In [ ]:
!pip install xgboost --quiet

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ======================================
# Load & Prepare Data
# ======================================
df = pd.read_csv("/content/drive/MyDrive/diabetes/Subject 25_3hr.csv")
cgm_col = [c for c in df.columns if 'cgm' in c.lower()][0]

# If time column exists
if 'EventDateTime' in df.columns:
    df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
    df = df.set_index('EventDateTime').sort_index()
    df = df.resample('15min').interpolate()

cgm = df[cgm_col].dropna().values.reshape(-1, 1)

print("\n✅ CGM Summary:")
print(df[cgm_col].describe())

# ======================================
# Split + Scale
# ======================================
train_size = int(len(cgm) * 0.8)
train, test = cgm[:train_size], cgm[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# ======================================
# Create Sequences
# ======================================
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

lookback = 60
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ======================================
# LSTM Model
# ======================================
lstm = Sequential([
    LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    LSTM(64),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

lstm_train = lstm.predict(X_train)
lstm_test = lstm.predict(X_test)

# ======================================
# GRU Model
# ======================================
gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(lookback, 1)),
    Dropout(0.3),
    GRU(64),
    Dense(1)
])
gru.compile(optimizer='adam', loss='mse')
gru.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0,
         validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

gru_train = gru.predict(X_train)
gru_test = gru.predict(X_test)

# ======================================
# Meta Learners
# ======================================
X_train_meta = np.hstack([lstm_train, gru_train])
X_test_meta = np.hstack([lstm_test, gru_test])

meta_lr = LinearRegression().fit(X_train_meta, y_train)
meta_rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_meta, y_train.ravel())

pred_lr = scaler.inverse_transform(meta_lr.predict(X_test_meta).reshape(-1,1))
pred_rf = scaler.inverse_transform(meta_rf.predict(X_test_meta).reshape(-1,1))
y_true = scaler.inverse_transform(y_test.reshape(-1,1))

# ======================================
# Evaluate
# ======================================
for name, pred in zip(["Linear Regression", "Random Forest"], [pred_lr, pred_rf]):
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"\n📊 {name} Meta Results: RMSE={rmse:.4f}, R²={r2:.4f}")

plt.figure(figsize=(12,6))
plt.plot(y_true, label="Actual", color='blue')
plt.plot(pred_rf, label="Predicted (RF)", color='red')
plt.title("CGM Prediction - LSTM+GRU+RF Hybrid")
plt.legend()
plt.show()